# Copying dashboard filters to a new dashboard
This script assumes the following:
- You have a reference dashboard to copy filters from
- You want the filters you add to be linked to every element on your dashboard
- All your filters you're adding are standard string filters

This script can be modified to do the following:
- Avoid deleting every filter and readding it by changing the `do_not_remove_these_filters_list` variable in the `check_filter_in_do_not_remove_list` function

## Todo:
- If you want to add numeric filters or anything outside of standard string filters, they'll need to be tested prior to use

In [1]:
import looker_sdk
from looker_sdk.sdk.api40.models import WriteCreateDashboardFilter, ResultMakerFilterablesListen, DashboardFilter

In [2]:
sdk = looker_sdk.init40()

In [3]:
# This is the only cell you need to update

# UPDATE THE NUMBERS IN THESE VARIABLES
reference_dashboard_id = '99999999999' # The dashboard you're using as a base to copy filters from
dashboard_to_update_id_list = ['111111111', '222222222222'] # The dashboards you'd like to copy filters to

def check_filter_in_do_not_remove_list(filter_obj):
    '''
    Checks to see if a filter is in the list of filters that are actually parameters.
    '''
    
    # UPDATE THE FILTER NAMES IN THIS LIST, USE THE NAMES YOU SEE IN THE DASHBOARD, NOT THE FIELD NAMES
    do_not_remove_these_filters_list = ['UPDATE ME', 'UPDATE ME']
    return filter_obj.title in do_not_remove_these_filters_list

In [4]:



def check_and_remove_dashboard_filters(dashboard_to_update_filter):
    # check if the filter is in the param list, which means we shouldn't remove it
    if check_filter_in_do_not_remove_list(dashboard_to_update_filter):
        return ''
    # The filter is not in the do not remove list, so we need to remove it from the new dashboard
    sdk.delete_dashboard_filter(dashboard_to_update_filter.id)


def create_new_filter_obj(dashboard_to_update_id: str, reference_dashboard_filter_obj: DashboardFilter):
    '''
    dashboard_to_update_id: str -                       The ID of the dashbaord to update as a string
    reference_dashboard_filter_obj: DashboardFilter -   The entire filter object for the reference dashboard
    '''
    return WriteCreateDashboardFilter(
        dashboard_id=dashboard_to_update_id, # Note that this is the dashboard_to_update_id, not the reference
        name=reference_dashboard_filter_obj.name,
        title=reference_dashboard_filter_obj.title,
        type=reference_dashboard_filter_obj.type,
        default_value=reference_dashboard_filter_obj.default_value,
        model=reference_dashboard_filter_obj.model,
        explore=reference_dashboard_filter_obj.explore,
        dimension=reference_dashboard_filter_obj.dimension,
        row=reference_dashboard_filter_obj.row,
        listens_to_filters=reference_dashboard_filter_obj.listens_to_filters,
        allow_multiple_values=reference_dashboard_filter_obj.allow_multiple_values,
        required=reference_dashboard_filter_obj.required,
        ui_config=reference_dashboard_filter_obj.ui_config,
    )


def create_new_result_maker_filterables_obj(dashboard_filter_name, field):
    return ResultMakerFilterablesListen(
        dashboard_filter_name=dashboard_filter_name,
        field=field
    )

In [5]:
reference_dashboard_filters = sdk.dashboard_dashboard_filters(str(reference_dashboard_id))

In [6]:
# This for loop is going to add only net-new filters. 
# If the filter already exists or is in the list of , it will leave it alone
for dashboard_to_update_id in dashboard_to_update_id_list:
    dashboard_to_update_id = str(dashboard_to_update_id)
    # Get the filters for the new dashboard
    dashboard_to_update_filters = sdk.dashboard_dashboard_filters(dashboard_to_update_id)
    # Get the elements for the new dashboard, we'll need these later
    dashboard_to_update_elements = sdk.dashboard_dashboard_elements(dashboard_to_update_id)

    # Remove filters
    for dashboard_to_update_filter in dashboard_to_update_filters:
        check_and_remove_dashboard_filters(dashboard_to_update_filter)

    # This is where we actually add the filters from the reference dashboard to the new dashboard
    for reference_dashboard_filter in reference_dashboard_filters:
        # check if the filter is in the do_not_remove_list, because then we don't need to do anything
        if check_filter_in_do_not_remove_list(reference_dashboard_filter):
            continue
        # Create the new filter object, note that all that's changing here is the dashboard_id attribute
        write_create_filter_obj = create_new_filter_obj(dashboard_to_update_id=dashboard_to_update_id, reference_dashboard_filter_obj=reference_dashboard_filter)
        # If the filter already exists, we can't create it, so it will throw an error
        try:
            new_filter = sdk.create_dashboard_filter(write_create_filter_obj)
        # The filter already exists
        except:
            continue
    
    # Now go through all the elements in the dashboard that was updated above and connect them to the new filters that were made
    # Note this will only add the filter you added above
        for element in dashboard_to_update_elements:
            # Check if this is a vis tile, we don't want to update text tiles because they don't need to be connected to filters
            if element.type != 'vis':
                continue

            # Copy the element to a new variable
            updated_element = element

            # Make the new ResultMakerFilterable object using the new_filter created above
            new_result_maker_filterable = create_new_result_maker_filterables_obj(
                dashboard_filter_name=new_filter.name,
                field=new_filter.dimension
            )

            # Append the new filter to the list of filters to listen to for this updated_element
            updated_element.result_maker.filterables[0].listen.append(new_result_maker_filterable)

            # update the element, we could technically use the updated_element id here, but sticking to the original for fun
            sdk.update_dashboard_element(element.id, body={'result_maker': updated_element.result_maker})
    print(f'Finished with Dashboard {dashboard_to_update_id}')

